In [ ]:
import pandas as pd
import numpy as np
import os
from os import listdir
from datetime import datetime
from time import mktime

In [ ]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

dfMalware=pd.DataFrame()
for idx,file in enumerate(find_csv_filenames('./USTC-TFC2016/Malware')):
    print(file,idx)
    if dfMalware.empty:
        df=pd.read_csv('./USTC-TFC2016/Malware/'+file)
        df=pd.concat([df,pd.DataFrame({"label":[idx+1 for i in range(len(df))]})],axis=1)
        dfMalware=df
    else:
        df=pd.read_csv('./USTC-TFC2016/Malware/'+file)
        df=pd.concat([df,pd.DataFrame({"label":[idx+1 for i in range(len(df))]})],axis=1)
        dfMalware=pd.concat([dfMalware,df])
        
dfBenign=pd.DataFrame()
for file in find_csv_filenames('./USTC-TFC2016/Benign'):
    print(file,idx)
    if dfBenign.empty:
        df=pd.read_csv('./USTC-TFC2016/Benign/'+file)
        df=pd.concat([df,pd.DataFrame({"label":[0 for i in range(len(df))]})],axis=1)
        dfBenign=df
    else:
        df=pd.read_csv('./USTC-TFC2016/Benign/'+file)
        df=pd.concat([df,pd.DataFrame({"label":[0 for i in range(len(df))]})],axis=1)
        dfBenign=pd.concat([dfBenign,df])
netflow=pd.concat([dfBenign,dfMalware])

In [ ]:
conn_order = ["ts", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes",	"tunnel_parents"]

data={}
data_mal={}
for d in os.listdir("./zeek/Benign"):
    f=open("./zeek/Benign/"+d+"/conn.log",'r')
    lines = f.readlines()
    data[d]=[]
    for line in lines[8:-1]:
        details = line.split('	')
        details = [x.strip() for x in details]
        structure = {key: value for key, value in zip(conn_order, details)}
        data[d].append(structure)
        
for d in os.listdir("./zeek/Malware"):
    f=open("./zeek/Malware/"+d+"/conn.log",'r')
    lines = f.readlines()
    data_mal[d]=[]
    for line in lines[8:-1]:
        details = line.split('	')
        details = [x.strip() for x in details]
        structure = {key: value for key, value in zip(conn_order, details)}
        data_mal[d].append(structure)
data_labeled=[]
for d in data.keys():
    df_temp = pd.DataFrame(data[d], columns=data[d][0].keys())
    anomaly = [1] * len(df_temp.index)
    df_temp['anomaly'] = anomaly 
    data_labeled.append(df_temp)

df = pd.concat(data_labeled)

data_mal_labeled=[]
for d in data_mal.keys():
    df_temp = pd.DataFrame(data_mal[d], columns=data_mal[d][0].keys())
    anomaly = [-1] * len(df_temp.index)
    df_temp['anomaly'] = anomaly 
    data_mal_labeled.append(df_temp)
    
df_anomaly = pd.concat(data_mal_labeled)
zeek = pd.concat([df, df_anomaly])
    
#zeek = pd.DataFrame(zeek_data, columns=zeek_data[0].keys())

In [ ]:
print(zeek.columns)
print(netflow.columns)
#print(netflow.dir.value_counts())

In [ ]:
zeek.duration=pd.to_numeric(zeek.duration,errors='coerce')
zeek=zeek.dropna()
#zeek.ts=zeek.ts.astype(float)#+zeek.duration
#zeek.ts=zeek.ts-zeek.ts.iloc[0]
#zeek=zeek.sort_values('ts')
#print(zeek.ts)

netflow=netflow.dropna()
netflow.dp=netflow.dp.astype(int)
#netflow['ts'] =netflow['ts'].apply(lambda x: mktime(datetime.strptime(x,'%Y-%m-%d %H:%M:%S').timetuple()))
#netflow.ts=netflow.ts#+netflow.td.astype(float)
#netflow.ts=netflow.ts-netflow.ts.iloc[0]
#netflow=netflow.sort_values('ts')
#print(netflow.ts)

In [ ]:
print(netflow)
print(zeek)

In [ ]:
netflow.sa=netflow.sa.astype(str)
netflow.da=netflow.da.astype(str)
netflow.sp=netflow.sp.astype(int)
netflow.dp=netflow.dp.astype(int)
netflow.dp=netflow.dp.astype(int)
netflow.ipkt=netflow.ipkt.astype(int)
netflow.opkt=netflow.opkt.astype(int)

zeek['id.orig_h']=zeek['id.orig_h'].astype(str)
zeek['id.resp_h']=zeek['id.resp_h'].astype(str)
zeek['id.orig_p']=zeek['id.orig_p'].astype(int)
zeek['id.resp_p']=zeek['id.resp_p'].astype(int)
zeek['orig_pkts']=zeek['orig_pkts'].astype(int)
zeek['resp_pkts']=zeek['resp_pkts'].astype(int)
netflow.sp=netflow.sp.astype(int)
netflow.dp=netflow.dp.astype(int)
netflow_to_zeek={"sa": "id.orig_h", "da": "id.resp_h","sp":"id.orig_p","dp":"id.resp_p"}
netflow=netflow.rename(columns=netflow_to_zeek)
common_cols = ['id.orig_h','id.resp_h','id.orig_p','id.resp_p']#,'orig_pkts','resp_pkts'] 
#netflow=netflow.drop_duplicates(subset=common_cols,keep='last')
zeek=zeek.drop_duplicates(subset=common_cols,keep='last')
df12 = pd.merge(netflow, zeek, on=common_cols, how='left')     #extract common rows with merge
#df12=df12.dropna()
df2 = zeek[~zeek['uid'].isin(df12['uid'])]
#print(df12.columns)
df3 = netflow[~netflow['ts'].isin(df12['ts_x'])]
print(df12.shape)
print(df2.shape)
print(df3.shape)
'''print(df2.iloc[0])
#print(df3.iloc[0])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df2[['ts']+common_cols])
print(df3[['ts']+common_cols])'''
zeek_to_netflow={v: k for k, v in netflow_to_zeek.items()}
df12=df12.rename(columns=zeek_to_netflow)
df12=df12.drop(['ts_y'],axis=1)
df12=df12.rename(columns={'ts_x':'ts'})
netflow=netflow.rename(columns=zeek_to_netflow)

In [ ]:
print(df12[netflow.columns.to_list()+['history']])

In [ ]:
df12.to_csv('./merged_ustc.csv',index=False)

In [ ]:
print(len(df12))
print(df12['history'].isna().sum())
print(df12.isna())